# Thermodynamic Relations

In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all
from nbimports import * 
from mmf_hfb import tf_completion as tf
from mmf_hfb import homogeneous
from scipy.optimize import brentq
from functools import partial
import itertools
tf.MAX_DIVISION = 500

In [ ]:
class FFState(object):
    def __init__(self, mu=10, dmu=0.4, delta=1,
                 m=1, T=0, hbar=1, k_c=100, d=2, fix_g=True):
        """      
        Arguments
        ---------
        fix_g : bool
           If fix_g is False, the class will fix C_tilde when
           performing the non-linear iterations, otherwise the
           coupling constant g_c will be fixed.  Note: this g_c will
           depend on the cutoff k_c.
        """
        self.fix_g = fix_g
        self.d = d
        self.T = T
        self.mu = mu
        self.m = m
        self.dmu = dmu
        self.delta = delta
        self.hbar = hbar
        self.k_c = k_c
        self._tf_args = dict(m_a=1, m_b=1, d=d, hbar=hbar, T=T, k_c=k_c)

        if fix_g:
            self._g = self.get_g(mu=mu, dmu=0, delta=delta, r=np.inf)
        else:
            self._C = tf.compute_C(mu_a=mu, mu_b=mu, delta=delta,
                                   **self._tf_args).n
        
    def f(self, mu, dmu, delta, q=0, dq=0, **kw):
        args = dict(self._tf_args)
        args.update(kw)

        if self.fix_g:
            return self.get_g(mu=mu, dmu=dmu, q=q, dq=dq, delta=delta, **args) - self._g

        return tf.compute_C(delta=delta, q=q, dq=dq **args).n - self._C

    def get_g(self, delta, mu=None, dmu=None, q=0, dq=0, **kw):
        args = dict(self._tf_args, q=q, dq=dq, delta=delta)
        if mu is not None:
            args.update(mu_a=mu+dmu, mu_b=mu-dmu)
        nu_delta = tf.integrate_q(tf.nu_delta_integrand, **args)
        g = 1./nu_delta.n
        return g

    def get_densities(self, mu, dmu, q=0, dq=0, delta=None, k_c=None):
        if delta is None:
            delta = self.solve(mu=mu, dmu=dmu, q=q, dq=dq, 
                               a=self.delta * 0.8, b=self.delta * 1.2)
        args = dict(self._tf_args, mu_a=mu + dmu, mu_b=mu - dmu, delta=delta,
                    q=q, dq=dq)
        if k_c is not None:
            args['k_c'] = k_c
            
        n_p = tf.integrate_q(tf.n_p_integrand, **args)
        n_m = tf.integrate_q(tf.n_m_integrand, **args)
        n_a, n_b = (n_p + n_m)/2, (n_p - n_m)/2
        return n_a, n_b
    
    def get_energy_density(self, mu, dmu, q=0, dq=0, delta=None,
                           n_a=None, n_b=None):
        if delta is None:
            delta = self.solve(mu=mu, dmu=dmu, q=q, dq=dq, 
                               a=self.delta * 0.8, b=self.delta * 1.2)
        if n_a is None:
            n_a, n_b = self.get_densities(mu=mu, dmu=dmu, delta=delta, q=q, dq=dq)

        args = dict(self._tf_args, mu_a=mu + dmu, mu_b=mu - dmu, delta=delta,
                    q=q, dq=dq)
        kappa = tf.integrate_q(tf.kappa_integrand, **args)
        if self.fix_g:
            g_c = self._g
        else:
            g_c = 1./self._C
        return kappa #  - g_c * n_a * n_b /2
    
    def get_pressure(self, mu, dmu, q=0, dq=0, delta=None, return_ns = False):
        if delta is None:
            delta = self.solve(mu=mu, dmu=dmu, q=q, dq=dq, 
                               a=self.delta * 0.1, b=self.delta * 2)
        n_a, n_b = self.get_densities(mu=mu, dmu=dmu, delta=delta, q=q, dq=dq)
        energy_density = self.get_energy_density(
            mu=mu, dmu=dmu, delta=delta, q=q, dq=dq,
            n_a=n_a, n_b=n_b)
        mu_a, mu_b = mu + dmu, mu - dmu
        pressure = mu_a * n_a + mu_b * n_b - energy_density
        if return_ns:
            return (pressure, n_a, n_b)
        return pressure

    def solve(self, mu=None, dmu=None, q=0, dq=0, a=0.8, b=1.2):
        args = dict(self._tf_args, q=q, dq=dq)
        if mu is not None:
            args.update(mu=mu, dmu=dmu, mu_a=mu+dmu, mu_b=mu-dmu)
        def f(delta):
            if self.fix_g:
                return self._g - self.get_g(delta=delta, mu=mu, dmu=dmu, q=q, dq=dq)
            return self._C - tf.compute_C(delta=delta, **args).n
        try:
            delta = brentq(f, a, b)
        except:
            delta = 0
        return delta

## Test Code
* The $\Delta_0$ is set to unitary, which may be not good for large $mu$ and $dmu$

In [ ]:
def test_Thermodynamic(mu, dmu,delta0=1, dim=1, k_c=100, q=0, dq=0):
    print(f"mu={mu}\tdmu={dmu}\tkc={k_c}\tq={q}\tdq={dq}\tdim={dim}")
    dx = 1e-3
    ff = FFState(dmu=dmu, mu=mu, delta=delta0, d=dim, k_c=k_c, fix_g=True)
    
    def get_P(mu, dmu):
        delta = ff.solve(mu=mu, dmu=dmu, q=q, dq=dq)
        return ff.get_pressure(mu=mu, dmu=dmu, delta=delta, q=q, dq=dq)

    def get_E_n(mu, dmu):
        E = ff.get_energy_density(mu=mu, dmu=dmu, q=q, dq=dq)
        n = sum(ff.get_densities(mu=mu, dmu=dmu, q=q, dq=dq))
        return E, n

    def get_ns(mu, dmu):
        return ff.get_densities(mu=mu, dmu=dmu, q=q, dq=dq)
      
    E1, n1 = get_E_n(mu=mu+dx, dmu=dmu)
    E0, n0 = get_E_n(mu=mu-dx, dmu=dmu)

    n_p = (get_P(mu+dx, dmu) - get_P(mu-dx, dmu))/2/dx
    n_a, n_b = get_ns(mu, dmu)
    n_a_ = (get_P(mu+dx/2, dmu+dx/2) - get_P(mu-dx/2, dmu - dx/2))/2/dx
    n_b_ = (get_P(mu+dx/2, dmu-dx/2) - get_P(mu-dx/2, dmu + dx/2))/2/dx
    print(f"n_a={n_a.n}\tNumerical  n_a={n_a_.n}")
    print(f"n_b={n_b.n}\tNumerical  n_b={n_b_.n}")
    print(f"n_p={n_a.n+n_b.n}\tNumerical  n_p={n_p.n}")
    print(f"mu={mu}\tNumerical mu={((E1-E0)/(n1-n0)).n}")
    assert np.allclose(n_a.n, n_a_.n, rtol=1e-4)
    assert np.allclose(n_b.n, n_b_.n, rtol=1e-4)
    assert np.allclose(n_p.n, (n_a+n_b).n, rtol=1e-4)
    assert np.allclose(mu,((E1-E0)/(n1-n0)).n, rtol=1e-2)
    
def homogeneous_Density(mu, dmu, delta, dim):
    """return density,used for reference"""
    return homogeneous.Homogeneous(dim = dim).get_ns(mus_eff=(mu+dmu, mu-dmu), delta=delta, N_twist=12)

## 1D Cases 

In [ ]:
homogeneous_Density(mu=5, dmu=0.5, delta=1, dim=1)

### Good accurary for $\mu_+=\frac{\partial E}{\partial n_+}$

In [ ]:
test_Thermodynamic(mu=5, dmu=.5, k_c=200, q=0, dq=0, dim=1, delta0=1)

In [ ]:
test_Thermodynamic(mu=5, dmu=2.5, k_c=100, q=.5, dq=1.0, dim=1, delta0=1)

### Medium accurary

In [ ]:
test_Thermodynamic(mu=5, dmu=.5, k_c=200, q=2.0, dq=1.0, dim=1, delta0=1)

In [ ]:
test_Thermodynamic(mu=5, dmu=.5, k_c=200, q=1.0, dq=1.0, dim=1, delta0=1)

## Poor accurary for  $\mu_+=\frac{\partial E}{\partial n_+}$

In [ ]:
homogeneous_Density(mu=5, dmu=2.5, delta=5, dim=1)

In [ ]:
test_Thermodynamic(mu=5, dmu=2.5, k_c=100, q=0, dq=0, dim=1, delta0=1)

In [ ]:
test_Thermodynamic(mu=5, dmu=2.5, k_c=500, q=0, dq=0, dim=1, delta0=5)

### $q$ and $dq$ do change the density

In [ ]:
test_Thermodynamic(mu=5, dmu=2.5, k_c=100, q=.5, dq=1.0, dim=1, delta0=1)

# 2D Cases

In [ ]:
homogeneous_Density(mu=5, dmu=0.64, delta=1, dim=2)

### Good accuracy

In [ ]:
test_Thermodynamic(mu=10, dmu=0.64, dim=2, q=0, dq=0, delta0=1, k_c=200)

### A local maximum in pressure with $dq \ne 0$

In [ ]:
qs = [0.5]#np.linspace(0,3,4) # q does not change the pressure, you may use q = 1.5 or other values 
dq0 = 7.856742013183863e-5
mu=0.5
dmu = 0.00039283710065919316
delta = 0.0007071067811865476
for q in qs:
    dqs = np.linspace(2,7,10)*dq0
    ff = FFState(mu=mu, dmu=dmu, d=2, delta=delta, k_c=500)
    ps = [ff.get_pressure(mu=mu, dmu=dmu,q = q, dq=dq).n for dq in dqs]
    clear_output()
    plt.plot(dqs * 1000, ps)

# 3D Cases:
* the 3D cases are very sensitive to the cutoff k_c

### homogeneous density

In [ ]:
homogeneous_Density(mu=5, dmu=0.64, delta=1, dim=3)

### Good accuracy

In [ ]:
test_Thermodynamic(mu = 5, dmu = 0.64, dim = 3, k_c = 250, q = 0, dq = 0)

### Poor density accurary

In [ ]:
test_Thermodynamic(mu = 5, dmu = 0.64, dim = 3, k_c = 500, q = 0, dq = 0)